# 第2章: 明確で直接的であること

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## セットアップ

次のセットアップセルを実行して、APIキーをロードし、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
from botocore.exceptions import ClientError
import json

# utilsパッケージからhintsモジュールをインポート
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None):
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 3000
    }
    additional_model_fields = {
        "top_p": 1
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config,
        "additionalModelRequestFields": additional_model_fields
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"クライアントエラーが発生しました: {message}")

---

## レッスン

**Claudeは明確で直接的な指示に最もよく反応します。**

Claudeを新しい仕事を始めた他の人のように考えてください。**Claudeは何をすべきかについての文脈を持っていません**。あなたが文字通り伝えること以外には。人に初めてタスクを指示する時と同じように、Claudeに対してあなたが望むことを明確に説明すればするほど、Claudeの反応はより良く、より正確になります。

疑問がある場合は、**明確なプロンプトの黄金ルール**に従ってください：
- あなたのプロンプトを同僚や友人に見せて、彼ら自身が指示に従って結果を出せるかどうかを確認してください。もし彼らが混乱しているなら、Claudeも混乱しています。

### 例

詩を書くというタスクを考えてみましょう。（音節の不一致は無視してください - LLMはまだ音節を数えるのが得意ではありません。）

In [ ]:
# プロンプト
PROMPT = "Write a haiku about robots."

# Claudeの応答を出力します
print(get_completion(PROMPT))

この俳句は十分に素敵ですが、ユーザーは「ここに俳句があります」という前置きなしに、Claudeに直接詩に入ってほしいかもしれません。

それをどうやって実現しますか？私たちは**それを頼みます**！

In [ ]:
# プロンプト
PROMPT = "ロボットについての俳句を書いてください。前置きは省略し、詩に直接入ってください。"

# クロードの応答を出力
print(get_completion(PROMPT))

以下の例を見てみましょう。クロードに「史上最高のバスケットボール選手は誰か」と尋ねてみましょう。下に示すように、クロードはいくつかの名前を挙げていますが、**明確な「最高」とは答えていません**。

In [ ]:
# プロンプト
PROMPT = "Who is the best basketball player of all time?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

クラウドに決断を下して最優秀選手を選んでもらうことはできますか？はい！ただ聞いてみてください！

In [ ]:
# プロンプト
PROMPT = "Who is the best basketball player of all time? Yes, there are differing opinions, but if you absolutely had to pick one player, who would it be?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

内容を変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして、[**Example Playground**](#example-playground)にアクセスしてください。

---

## 演習
- [演習 2.1 - スペイン語](#exercise-21---spanish)
- [演習 2.2 - 一人プレイヤーのみ](#exercise-22---one-player-only)
- [演習 2.3 - 物語を書く](#exercise-23---write-a-story)

### 演習 2.1 - スペイン語
`SYSTEM_PROMPT`を修正して、Claudeがその回答をスペイン語で出力するようにします。

In [ ]:
# System prompt - this is the only field you should chnage
SYSTEM_PROMPT = "[このテキストを置き換えてください]"

# Prompt
PROMPT = "Hello Claude, how are you?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return "hola" in text.lower()

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_2_1_hint)

### 演習 2.2 - 一人プレイヤーのみ

`PROMPT`を修正して、Claudeが全く曖昧さを持たず、**他の言葉や句読点なしで**、特定の一人のプレイヤーの名前だけを返すようにします。

In [ ]:
# Prompt - これは変更すべき唯一のフィールドです
PROMPT = "[このテキストを置き換えてください]"

# Claudeの応答を取得
response = get_completion(PROMPT)

# 演習の正確さを評価する関数
def grade_exercise(text):
    return text == "Michael Jordan"

# Claudeの応答と対応する評価を出力
print(response)
print("\n--------------------------- 評価 ---------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_2_2_hint)

### 演習 2.3 - 物語を書く

`PROMPT`を修正して、Claudeができるだけ長い回答をするようにしてください。あなたの回答が**800語を超える**場合、Claudeの回答は正しいと評価されます。

In [ ]:
# Prompt - これは変更すべき唯一のフィールドです
PROMPT = "[Replace this text]"

# Claudeの応答を取得
response = get_completion(PROMPT)

# 演習の正確さを評価する関数
def grade_exercise(text):
    trimmed = text.strip()
    words = len(trimmed.split())
    return words >= 800

# Claudeの応答と対応する評価を出力
print(response)
print("\n--------------------------- 評価 ---------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_2_3_hint)

### おめでとうございます！

ここまでのすべての演習を解決した場合、次の章に進む準備が整いました。楽しいプロンプト作成を！

---

## 例のプレイグラウンド

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのように影響するかを見るためのエリアです。

In [ ]:
# プロンプト
PROMPT = "Write a haiku about robots."

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "ロボットについての俳句を書いてください。前置きは省略し、詩に直接入ってください。"

# クロードの応答を出力
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "Who is the best basketball player of all time?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "Who is the best basketball player of all time? Yes, there are differing opinions, but if you absolutely had to pick one player, who would it be?"

# Claudeの応答を出力します
print(get_completion(PROMPT))